# CDC Vaccine Data Pipeline

This notebook extracts vaccination provider locations and flu coverage data from CDC APIs,
performs data cleaning and geographic joining, and saves the combined dataset for analysis.

Author: Nathaniel Pearson

Date: August 2025


In [1]:
import requests
import pandas as pd
import numpy as np
import json
from typing import Dict, List, Optional
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Loading data from CDC Socrata API with paginatio
def load_cdc_data(url, limit=50000):
    all_data = []
    offset = 0
    
    print(f"Loading data from: {url}")
    
    while True:
        params = {
            "$limit": limit,
            "$offset": offset,
            "$order": ":id"
        }
        
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise error if request fails
        batch_data = response.json()
        if not batch_data:
            break
        all_data.extend(batch_data)
        print(f"  Loaded {len(batch_data)} records (total: {len(all_data)})")
        if len(batch_data) < limit:
            break
        offset += limit
        time.sleep(1)
    
    print(f"Total records: {len(all_data)}")
    
    # Conversion to DataFrame
    df = pd.DataFrame(all_data)
    return df


# Loading datasets
provider_url = "https://data.cdc.gov/resource/bugr-bbfr.json"
coverage_url = "https://data.cdc.gov/resource/vh55-3he6.json"
providers_df = load_cdc_data(provider_url)
coverage_df = load_cdc_data(coverage_url)

print(f"\nProviders dataset: {providers_df.shape}")
print(f"Coverage dataset: {coverage_df.shape}")
print("\nProviders columns:")
print(providers_df.columns.tolist())
print("\nCoverage columns:")
print(coverage_df.columns.tolist())

Loading data from: https://data.cdc.gov/resource/bugr-bbfr.json
  Loaded 50000 records (total: 50000)
  Loaded 50000 records (total: 100000)
  Loaded 50000 records (total: 150000)
  Loaded 50000 records (total: 200000)
  Loaded 2652 records (total: 202652)
Total records: 202652
Loading data from: https://data.cdc.gov/resource/vh55-3he6.json
  Loaded 50000 records (total: 50000)
  Loaded 50000 records (total: 100000)
  Loaded 50000 records (total: 150000)
  Loaded 50000 records (total: 200000)
  Loaded 20729 records (total: 220729)
Total records: 220729

Providers dataset: (202652, 28)
Coverage dataset: (220729, 11)

Providers columns:
['provider_location_guid', 'loc_store_no', 'loc_phone', 'loc_name', 'loc_admin_street1', 'loc_admin_street2', 'loc_admin_city', 'loc_admin_state', 'loc_admin_zip', 'sunday_hours', 'monday_hours', 'tuesday_hours', 'wednesday_hours', 'thursday_hours', 'friday_hours', 'saturday_hours', 'insurance_accepted', 'walkins_accepted', 'provider_notes', 'searchable_n

In [3]:
coverage_df

,vaccine,geography_type,geography,fips,year_season,month,dimension_type,dimension,coverage_estimate,_95_ci,population_sample_size
0,Seasonal Influenza,Counties,New Haven,09009,2018,1,>=18 Years,Non-Medical Setting,45.5,43.9 to 47.2,NaN
1,Seasonal Influenza,Counties,New Haven,09009,2021,1,>=18 Years,Non-Medical Setting,53.0,46.0 to 60.9,NaN
2,Seasonal Influenza,Counties,New Haven,09009,2020,1,Age,>=18 Years,52.4,50.6 to 54.3,NaN
3,Seasonal Influenza,Counties,New Haven,09009,2021,1,Age,>=18 Years,50.2,45.4 to 55.8,NaN
4,Seasonal Influenza,Counties,New Haven,09009,2018,1,Age,>=18 Years,34.0,32.6 to 35.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...
220724,Seasonal Influenza,States/Local Areas,Maine,23,2017-18,1,>=65 Years,Non-Medical Setting,41.4,36.8 to 46.0,NaN
220725,Seasonal Influenza,States/Local Areas,Maine,23,2020-21,1,>=65 Years,Non-Medical Setting,30.0,26.6 to 33.4,2064
220726,Seasonal Influenza,States/Local Areas,Maine,23,2017-18,1,18-64 Years,Non-Medical Setting,45.6,39.7 to 51.5,NaN
220727,Seasonal Influenza,States/Local Areas,Maine,23,2020-21,1,18-64 Years,Non-Medical Setting,33.7,29.7 to 37.7,2772


In [4]:
count = coverage_df[coverage_df['geography_type'] == 'Counties'].shape[0]
count

21818

# Data Manipulation

In [7]:
# Select useful columns from provders_df
joint_df = providers_df[['loc_name','loc_admin_city',"loc_admin_state", 
                        "loc_admin_zip", "insurance_accepted", "walkins_accepted", "supply_level", 
                        "latitude", "longitude",
                         'sunday_hours', 'monday_hours', 'tuesday_hours', 'wednesday_hours', 'thursday_hours', 'friday_hours', 'saturday_hours']]

In [8]:
# Select years 2023-2024 for coverage_df. Select 12th month to represent the end of the year
coverage_df_2023_to_2024 = coverage_df[
    coverage_df['year_season'].str.startswith(('2023'))
]
coverage_df_2023_to_2024['month'] = pd.to_numeric(coverage_df_2023_to_2024['month'], errors='coerce')
coverage_df_2023_to_2024 = coverage_df_2023_to_2024[coverage_df_2023_to_2024['month'] == 12]

In [10]:
# Calculation for the total operation hours based on weekday opening times
from datetime import datetime, timedelta

def parse_hours(hours_str):
    if pd.isna(hours_str) or str(hours_str).strip().lower() == 'closed':
        return 0.0
    try:
        open_time_str, close_time_str = [t.strip() for t in hours_str.split('-')]
        open_time = datetime.strptime(open_time_str, '%I:%M %p')
        close_time = datetime.strptime(close_time_str, '%I:%M %p')
        if close_time < open_time:
            close_time += timedelta(days=1)
        duration = (close_time - open_time).seconds / 3600
        return duration
    except Exception:
        return 0.0

days = ['sunday_hours', 'monday_hours', 'tuesday_hours', 'wednesday_hours',
        'thursday_hours', 'friday_hours', 'saturday_hours']

# joint_df is connected version of providers_df and coverage_df. 
joint_df['weekly_hours_open'] = joint_df[days].applymap(parse_hours).sum(axis=1)
joint_df = joint_df.drop(columns=days)

In [12]:
# Adding county to joint_df
zip_county_df = pd.read_csv('data/zipcode_county.csv')
joint_df['zip5'] = joint_df['loc_admin_zip'].str[:5]
zip_county_df['ZIP'] = zip_county_df['ZIP'].astype(str).str.zfill(5)
joint_df = joint_df.merge(zip_county_df[['ZIP', 'COUNTYNAME']], left_on='zip5', right_on='ZIP', how='left')

In [13]:
# State average

# convert state names in joint_df to full names
import us
abbr_to_name = {state.abbr: state.name for state in us.states.STATES}
joint_df['loc_admin_state'] = joint_df['loc_admin_state'].map(abbr_to_name)

#convert coverage_estimate to numeric
coverage_df_2023_to_2024['coverage_estimate'] = pd.to_numeric(
    coverage_df_2023_to_2024['coverage_estimate'].str.extract(r'([\d\.]+)')[0], errors='coerce'
)

# Calculate average coverage by state
state_avg = coverage_df_2023_to_2024.groupby('geography')['coverage_estimate'].mean().reset_index()
state_avg.rename(columns={'coverage_estimate': 'state_coverage'}, inplace=True)

# Merge into joint_df
joint_df = joint_df.merge(state_avg, left_on='loc_admin_state', right_on='geography', how='left')

# Drop the extra 'Geography' column
joint_df.drop(columns=['geography'], inplace=True)

#rename columns
joint_df.rename(columns={'ZIP': 'zip', 'COUNTYNAME': 'county_name'}, inplace=True)

In [14]:
# Convert coverage_estimate to numeric
coverage_df_2023_to_2024['coverage_estimate'] = pd.to_numeric(
    coverage_df_2023_to_2024['coverage_estimate'], errors='coerce'
)

# Filter for state-level data only
state_data = coverage_df_2023_to_2024[coverage_df_2023_to_2024['geography_type'] == 'States/Local Areas']

# Calculate mean coverage by state and dimension
grouped = state_data.groupby(['geography', 'dimension'])['coverage_estimate'].mean().reset_index()

# Pivot so each dimension becomes a separate column
pivot_df = grouped.pivot(index='geography', columns='dimension', values='coverage_estimate').reset_index()

# Rename columns: add 's.' prefix and format names
pivot_df.columns = ['loc_admin_state'] + ['s.' + col.lower().replace(' ', '_').replace(',', '').replace('-', '_') for col in pivot_df.columns[1:]]

# Merge the dimension averages into joint_df by state
joint_df = joint_df.merge(pivot_df, on='loc_admin_state', how='left')

In [15]:
joint_df.to_csv('joint_df_export.csv', index=False)

# County Calculations

In [158]:
county_df = coverage_df[coverage_df['geography_type'] == 'Counties']

In [159]:
county_df['coverage_estimate'] = pd.to_numeric(county_df['coverage_estimate'], errors='coerce')
avg_county_coverage = county_df.groupby('geography')['coverage_estimate'].mean().reset_index()

In [160]:
avg_county_coverage_by_dim = county_df.groupby(['geography', 'dimension'])['coverage_estimate'].mean().reset_index()
df_pivot = avg_county_coverage_by_dim.pivot(index='geography', columns='dimension', values='coverage_estimate').reset_index()
df_pivot = df_pivot.rename(columns={'geography': 'county_name'})

In [161]:
df_pivot

dimension,county_name,>=18 Years,Non-Medical Setting
0,Abbeville,42.24,43.4500
1,Acadia Parish,35.06,38.0500
2,Accomack,44.60,38.3500
3,Ada,44.04,54.4500
4,Adair,41.08,39.2625
...,...,...,...
1883,Yukon-Koyukuk Census Area,34.34,25.1500
1884,Yuma,37.99,44.8750
1885,Zapata,28.54,36.8500
1886,Zavala,28.66,35.8000


In [162]:
# Count locations per county
county_counts = joint_df.groupby('county_name').size().reset_index(name='location_count')
county_counts['county_name'] = county_counts['county_name'].str.split().str[0]

In [163]:
avg_county_coverage = avg_county_coverage.rename(columns={'geography': 'county_name'})
merged_df = avg_county_coverage.merge(county_counts, on='county_name', how='left')

merged_df

,county_name,coverage_estimate,location_count
0,Abbeville,42.585714,83.0
1,Acadia Parish,35.914286,NaN
2,Accomack,42.814286,18.0
3,Ada,47.014286,422.0
4,Adair,40.560714,76.0
...,...,...,...
1930,Yukon-Koyukuk Census Area,31.714286,NaN
1931,Yuma,39.957143,98.0
1932,Zapata,30.914286,7.0
1933,Zavala,30.700000,NaN


In [164]:
import censusdata

# Get 2022 ACS 5-year county population estimates
counties = censusdata.download('acs5', 2022, 
                              censusdata.censusgeo([('county', '*')]), 
                              ['B01003_001E'])  # Total population

In [165]:
counties = counties.reset_index().rename(columns={'index': 'county_name'})
counties = counties.rename(columns={'B01003_001E': 'population'})
counties['county_name'] = counties['county_name'].astype(str).str.split().str[0]

In [166]:
merged_df = merged_df.merge(counties[['county_name', 'population']], on='county_name', how='left')
merged_df

,county_name,coverage_estimate,location_count,population
0,Abbeville,42.585714,83.0,24368.0
1,Acadia Parish,35.914286,NaN,NaN
2,Accomack,42.814286,18.0,33367.0
3,Ada,47.014286,422.0,497494.0
4,Adair,40.560714,76.0,7479.0
...,...,...,...,...
3567,Yuma,39.957143,98.0,204374.0
3568,Yuma,39.957143,98.0,9938.0
3569,Zapata,30.914286,7.0,13896.0
3570,Zavala,30.700000,NaN,9700.0


In [167]:
merged_df = merged_df.merge(df_pivot, left_on='county_name', right_on='county_name', how='left')

merged_df.dropna(subset=['location_count'], inplace=True)
merged_df.isnull().sum()

county_name            0
coverage_estimate      0
location_count         0
population             3
>=18 Years             0
Non-Medical Setting    0
dtype: int64

In [168]:
merged_df

,county_name,coverage_estimate,location_count,population,>=18 Years,Non-Medical Setting
0,Abbeville,42.585714,83.0,24368.0,42.24,43.4500
2,Accomack,42.814286,18.0,33367.0,44.60,38.3500
3,Ada,47.014286,422.0,497494.0,44.04,54.4500
4,Adair,40.560714,76.0,7479.0,41.08,39.2625
5,Adair,40.560714,76.0,18887.0,41.08,39.2625
...,...,...,...,...,...,...
3564,Young,40.757143,30.0,17903.0,37.86,48.0000
3565,Yuba,34.657143,42.0,81705.0,34.48,35.1000
3567,Yuma,39.957143,98.0,204374.0,37.99,44.8750
3568,Yuma,39.957143,98.0,9938.0,37.99,44.8750


In [172]:
# Average out the values for all columns 
df_grouped = merged_df.groupby('county_name').agg({
    'coverage_estimate': 'mean',
    'location_count': 'mean',
    'population': 'mean',
    '>=18 Years': 'mean',
    'Non-Medical Setting' : 'mean'
}).reset_index()

df_grouped['location_per_1000'] = (df_grouped['location_count'] / df_grouped['population']) * 1000

df_grouped

,county_name,coverage_estimate,location_count,population,>=18 Years,Non-Medical Setting,location_per_1000
0,Abbeville,42.585714,83.0,24368.000000,42.240000,43.450000,3.406106
1,Accomack,42.814286,18.0,33367.000000,44.600000,38.350000,0.539455
2,Ada,47.014286,422.0,497494.000000,44.040000,54.450000,0.848251
3,Adair,40.560714,76.0,17847.750000,41.080000,39.262500,4.258240
4,Adams,40.990476,766.0,72155.083333,41.528333,39.645833,10.616023
...,...,...,...,...,...,...,...
1367,York,46.411429,1067.0,207435.800000,45.604000,48.430000,5.143760
1368,Young,40.757143,30.0,17903.000000,37.860000,48.000000,1.675697
1369,Yuba,34.657143,42.0,81705.000000,34.480000,35.100000,0.514044
1370,Yuma,39.957143,98.0,107156.000000,37.990000,44.875000,0.914554


In [173]:
df_grouped.to_csv('avg_county_coverage.csv', index=False)